# Zero shot learning - BART-LARGE MNLI

## Import Libraries

In [ ]:
import pandas as pd
import os
import numpy as np

from transformers import pipeline
from sklearn.model_selection import KFold

from sklearn.metrics import f1_score, accuracy_score

from tqdm.notebook import tqdm

from collections import Counter
from scipy import stats

In [ ]:
model = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0", device="cuda") 

In [8]:
data_path_open_source = "/home/neemias/PerceptSent-LLM-approach/data/"
# data_path_open_ai = "/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/"
data_path_open_ai = "/home/neemias/PerceptSent-LLM-approach/data/deepseek/"

In [ ]:
data_paths = [os.path.join(data_path_open_source, f) 
              for f in os.listdir(data_path_open_source) if f.endswith('.csv')] + [os.path.join(data_path_open_ai, f)
                                                              for f in os.listdir(data_path_open_ai) if f.endswith('.csv')]
del data_paths[6]
data_paths[-10:]

## Sentiment Analyzer

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for data_path in tqdm(data_paths[-10:]):
    print(data_path)
    df = pd.read_csv(data_path)
    print(len(df))
    df_metrics = pd.DataFrame([])
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(kfold.split(df)):
        train_df = pd.DataFrame({"text": df["text"].iloc[train_idx].to_list(), 
                                        "sentiment": df["sentiment"].iloc[train_idx].to_list()})
        val_df = pd.DataFrame({"text": df["text"].iloc[val_idx].to_list(), 
                                    "sentiment": df["sentiment"].iloc[val_idx].to_list()})
        

        text = val_df["text"].to_list()
        target = val_df["sentiment"].to_list()

        sentiments = np.unique(target)

        if (len(sentiments) == 3):
            sent_dic = {
                "Negative": 1,
                "Neutral": 0,
                "Positive": 2,
            }
            sents = ["Positive", "Neutral", "Negative"]
        elif (len(sentiments) == 2):
            # # P2+

            if (data_path.split('/')[-1].split('.')[0].split('_')[-1] == "p2plus"):
                sent_dic = {
                    "Negative": 1, "Positive": 0,
                }
            else:
                sent_dic = {
                    "Negative": 0, "Positive": 1,
                }
            sents = ["Negative", "Positive"]
        else:
            sent_dic = {
                "Positive": 4,
                "SlightlyPositive": 3,
                "Neutral": 2,
                "SlightlyNegative": 1,
                "Negative": 0
            }
            sents = ["Positive", "SlightlyPositive", "Neutral", "SlightlyNegative", "Negative"]

        pred = []
        for t in text:
            result = model(t, sents)
            pred.append(sent_dic[result["labels"][0]])
        accuracy_val = accuracy_score(target, pred)
        f1_val = f1_score(target, pred, average="weighted")    
        df_metrics = pd.concat([df_metrics, pd.DataFrame({"accuracy": [accuracy_val], "f1_score": [f1_val]
                                                            })], axis=0)
    # display(df_metrics.head())
    if (len(data_path.split('/')[-2].split('-')) > 2):
        flag = "openai"
    else:
        flag = "opensource"
    df_metrics.to_csv(f"/home/neemias/PerceptSent-LLM-approach/experiments/zero-shot-bart/{flag}-{data_path.split('/')[-1]}",
                      index=False)

In [ ]:
csv_files = [os.path.join("/home/neemias/PerceptSent-LLM-approach/experiments/zero-shot-bart", f) 
             for f in os.listdir("/home/neemias/PerceptSent-LLM-approach/experiments/zero-shot-bart") if f.endswith(".csv") and "opensource" in f]
csv_files.sort()
for csv_file in csv_files:
    df_metrics = pd.read_csv(csv_file)
    f1_scores = df_metrics["f1_score"].to_list()
    mean_f1 = np.mean(f1_scores)


    # define the confidence level
    confidence_level = 0.95
    degrees_freedon = len(f1_scores)-1

    confidence_interval = stats.t.interval(
        confidence_level, 
        degrees_freedon, 
        loc=mean_f1, 
        scale=stats.sem(f1_scores)
    )

    print(f"\n\nProblem: {csv_file.split('/')[-1].split('.')[0]}")
    print(f"Max F1-score: {max(f1_scores)}")
    print(f"Average F1-score: {mean_f1}")
    print(f"Confidence interval 95%: {confidence_interval}")
    print(f"Inteval: {abs(confidence_interval[0]-mean_f1)} - Interval: {abs(confidence_interval[1]-mean_f1)}")
    confidence_interval = stats.t.interval(
        confidence_level, 
        degrees_freedon, 
        loc=max(f1_scores), 
        scale=stats.sem(f1_scores)
    )

    print(f"Inteval: {abs(confidence_interval[0]-max(f1_scores))} - Interval: {abs(confidence_interval[1]-max(f1_scores))}")